### Loading data using data loader

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.1 MB/s eta 0:00:00


In [2]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.3 MB/s eta 0:00:00


In [3]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
dataset = load_dataset("McGill-NLP/FaithDial")

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
dataset.shape

{'test': (3539, 7),
 'test_random_split': (1716, 7),
 'test_topic_split': (1823, 7),
 'train': (18357, 7),
 'validation': (3417, 7),
 'valid_random_split': (1666, 7),
 'valid_topic_split': (1751, 7)}

In [7]:
dataset["train"]

Dataset({
    features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
    num_rows: 18357
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [9]:
from collections import Counter
from typing import List, Tuple
import datasets
import torch
from tqdm import tqdm

In [10]:
class DatasetMap():
  @staticmethod
  def faithdial_begin(example):
    # nr_answer = len(example["response"])
    return [example["knowledge"]], [example["history"][-1]], [example["original_response"]], [','.join(example["BEGIN"])]

  @staticmethod
  def faithdial_vrm(example):
    # nr_answer = len(example["response"])
    return [example["knowledge"]], [example["history"][-1]], [example["original_response"]], [','.join(example["VRM"])]

In [11]:
class DialogDataset(torch.utils.data.Dataset):
  def __init__(self, hf_dataset: datasets.arrow_dataset.Dataset, tokenizer, parser=None):
        self.tokenizer = tokenizer
        self.questions: List[str] = []
        self.answers: List[str] = []
        self.contexts: List[str] = []
        self.begin: List[str] = []
        # self.vrm: List[str] = []
        for row in tqdm(hf_dataset):
          _contexts, _questions, _answers, _begin = parser(row)
          self.contexts += _contexts
          self.questions += _questions
          self.answers += _answers
          self.begin += _begin
          # self.vrm += _vrm
          self.item_count: int = len(self.questions)

  def __len__(self):
      """Magic method over-ride for class lenght evaluation
      Returns:
          int: lenght of the object 
      """
      return self.item_count

  def __getitem__(self, index: int):
      """Magic method over-ride for class getitem method
      Args:
          index (int): index for identify question-context and answer example
      Returns:
          Tuple(str,str,str): (Context, Question, Answer)
      """
      return self.contexts[index], self.questions[index], self.answers[index], self.begin[index]

  def pack_minibatch(self, data: List[Tuple[str, str]]):
      """Pack mini-batch function
      Args:
          data (Tuple[List[str],List[str],List[str]]): (Contexts, Questions, Answers)
      Returns:
          Tuple[List[str],List[str],List[str]]: (Contexts, Questions, Answers)
      """
      return zip(*data)

  def __exact_match_score(self, prediction, ground_truth):
      """_summary_
      Args:
          prediction (_type_): _description_
          ground_truth (_type_): _description_
      Returns:
          _type_: _description_
      """
      if len(ground_truth) == len(prediction):
          if all(token1 == token2 for token1, token2 in zip(ground_truth,prediction)):
              return 1
      return 0

  def __f1_score(self, prediction_tokens, ground_truth_tokens):
      """_summary_
      Args:
          prediction (_type_): _description_
          ground_truth (_type_): _description_
      Returns:
          _type_: _description_
      """
      common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
      num_same = sum(common.values())
      if num_same == 0:
          return 0
      precision = 1.0 * num_same / len(prediction_tokens)
      recall = 1.0 * num_same / len(ground_truth_tokens)
      f1 = (2 * precision * recall) / (precision + recall)
      return f1

  def evaluate(self, predictions, gold_answers):
    f1 = exact_match = 0
    for ground_truths, prediction in tqdm(zip(gold_answers, predictions)):
        # Remove pad token
        tokens_to_remove = {
            self.tokenizer.pad_token_id,
            self.tokenizer.eos_token_id,
            self.tokenizer.bos_token_id,
            self.tokenizer.cls_token_id,
            self.tokenizer.sep_token_id,
            self.tokenizer.mask_token_id
          }
        prediction = list(filter(lambda token: token not in tokens_to_remove, prediction))
        ground_truths = list(filter(lambda token: token not in tokens_to_remove, ground_truths))
        f1 += self.__f1_score(prediction, ground_truths)
        exact_match += self.__exact_match_score(prediction, ground_truths)
    return 100*f1/len(predictions), 100*exact_match/len(predictions)

#### Using text to text custom pipeline for custom training loop

In [12]:
from torch.utils.data import DataLoader

In [13]:
train_set = DialogDataset(dataset['train'], tokenizer, DatasetMap.faithdial_begin)

100%|██████████| 18357/18357 [00:03<00:00, 6036.45it/s]


In [14]:
validation_set = DialogDataset(dataset['validation'], tokenizer, DatasetMap.faithdial_begin)

100%|██████████| 3417/3417 [00:00<00:00, 7401.59it/s]


In [15]:
import random
print(random.choice(train_set))
print(random.choice(validation_set))

("Mopeds by definition are driven by both an engine and bicycle pedals, but in common usage and in many jurisdictions the term ''moped'' is used for similar vehicles including a scooter.", "That's interesting. Do all mopeds only have two wheels?", 'The only requirement for something to be deemed a moped is that it has an engine and bicycle pedals so by that definition I guess you can have as many as youd like!', 'Hallucination,Entailment')
('Since the 1920s Jazz Age, jazz has become recognized as a major form of musical expression.', 'Are there different sub genres of Jazz?', "Not much info on that but jazz since the 1920's age jaz has been recognized as a major form of musical expression", 'Hallucination,Entailment')


In [16]:
BATCH_SIZE = 32

In [17]:
my_trainset_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE,
                                    num_workers=8, collate_fn=lambda data: train_set.pack_minibatch(data))
my_validation_dataloader = DataLoader(validation_set, batch_size=BATCH_SIZE,
                                        num_workers=8, collate_fn=lambda data: validation_set.pack_minibatch(data))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
train_set[0]

('Internet access was once rare, but has grown rapidly.',
 'Can you imagine the world without internet access?',
 "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'Hallucination')

### Model Initialization

In [19]:
# Initialize the T5 model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

### Training

In [20]:
import torch
from tqdm import tqdm

In [21]:
DEVICE = 'cuda:0'
NUM_TRAIN_EPOCHS = 10
MAX_INPUT_LENGTH = 256

In [22]:
model.train()
model.to(DEVICE)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [23]:
# Initialize the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [24]:
for epoch in range(NUM_TRAIN_EPOCHS):
    epoch_train_loss = 0.
    ### Training Piece
    for contexts, questions, answers, begin in tqdm(my_trainset_dataloader):
        optimizer.zero_grad()
        inputs = list(map(lambda t: f"context: {t[0]}  question: {t[1]}  answer: {t[2]}", zip(contexts, questions, answers)))
        # targets = list(map(lambda t: f"begin: {t[0]}  vrm: {t[1]}", zip(begin, vrm)))
        encoded_inputs = tokenizer(
                                inputs,
                                padding="longest",
                                max_length=MAX_INPUT_LENGTH,
                                truncation=True,
                                return_tensors="pt",
                            )
        
        encoded_targets = tokenizer(
                                begin,
                                padding="longest",
                                max_length=10,
                                truncation=True,
                                return_tensors="pt",
                            )

        input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
        encoded_targets = encoded_targets.input_ids

        # replace padding target token id's of the labels by -100, crossEntropy skip target label == -100
        encoded_targets[encoded_targets == tokenizer.pad_token_id] = -100

        input_ids = input_ids.to(DEVICE)
        encoded_targets = encoded_targets.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=encoded_targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() * BATCH_SIZE
    print(f"epoch={epoch + 1}/{NUM_TRAIN_EPOCHS}")
    print(f"\t Train loss = {epoch_train_loss/len(train_set):.4f}")

    ## Validation Piece
    model.eval()
    epoch_val_loss = 0.
    with torch.no_grad():
        # model_predictions_encoded = []
        # target_encoded = []
        for contexts, questions, answers, begin in tqdm(my_validation_dataloader):
            inputs = list(map(lambda t: f"context: {t[0]}  question: {t[1]}  answer: {t[2]}", zip(contexts, questions, answers)))
            # targets = list(map(lambda t: f"begin: {t[0]}  vrm: {t[1]}", zip(begin, vrm)))
            encoded_inputs = tokenizer(
                inputs,
                padding="longest",
                max_length=MAX_INPUT_LENGTH,
                truncation=True,
                return_tensors="pt",
            )
            encoded_targets = tokenizer(
                begin,
                padding="longest",
                max_length=10,
                truncation=True,
                return_tensors="pt",
            )
            encoded_inputs, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
            encoded_targets = encoded_targets.input_ids

            encoded_inputs = encoded_inputs.to(DEVICE)
            encoded_targets = encoded_targets.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            outputs = model(input_ids=encoded_inputs, attention_mask=attention_mask, labels=encoded_targets)
            loss = outputs.loss
            epoch_val_loss += loss.item() * BATCH_SIZE
    print(f"\t Val loss = {epoch_val_loss/len(validation_set):.4f}")
    model.train()

100%|██████████| 574/574 [02:38<00:00,  3.62it/s]


epoch=1/10
	 Train loss = 0.2731


100%|██████████| 107/107 [00:10<00:00, 10.67it/s]


	 Val loss = 5.4031


100%|██████████| 574/574 [02:37<00:00,  3.65it/s]


...

	 Val loss = 6.4214


100%|██████████| 574/574 [02:36<00:00,  3.66it/s]


epoch=10/10
	 Train loss = 0.0500


100%|██████████| 107/107 [00:09<00:00, 10.77it/s]

	 Val loss = 6.6902


In [25]:
model.save_pretrained("only_begin")

### Inferencing

In [26]:
test_sample = dataset['test'][20]

In [27]:
test_sample

{'dialog_idx': 4,
 'response': 'Nice, did you know that elements of game design can also be applied to other interactions?',
 'original_response': 'Same and Increasingly, elements and principles of game design are also applied to other interactions',
 'history': ['I design video games for a living. Do you play any video games?',
  "I don't play any, but I do know about game design. It involves applying aesthetics and design to create a game.",
  'Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.'],
 'knowledge': 'Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).',
 'BEGIN': ['Hallucination', 'Entailment'],
 'VRM': ['Disclosure', 'Ack.']}

In [28]:
test_sample

{'dialog_idx': 4,
 'response': 'Nice, did you know that elements of game design can also be applied to other interactions?',
 'original_response': 'Same and Increasingly, elements and principles of game design are also applied to other interactions',
 'history': ['I design video games for a living. Do you play any video games?',
  "I don't play any, but I do know about game design. It involves applying aesthetics and design to create a game.",
  'Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.'],
 'knowledge': 'Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).',
 'BEGIN': ['Hallucination', 'Entailment'],
 'VRM': ['Disclosure', 'Ack.']}

In [29]:
def infer(model, prompt, knowledge, orig_resp, max_length=100):
    # question = "What is 42?"
    # context = "42 is the answer to life, the universe and everything"
    input = f"question: {prompt} context: {knowledge} answer: {orig_resp}"
    encoded_input = tokenizer([input],
                                return_tensors='pt',
                                max_length=512,
                                truncation=True)
    inp = encoded_input.input_ids.to(DEVICE)
    attn_mask = encoded_input.attention_mask.to(DEVICE)
    output = model.generate(input_ids = inp,
                                attention_mask = attn_mask, max_length=max_length)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    # print(output)
    return output

In [30]:
targets = infer(model, prompt=test_sample['history'][-1], knowledge=test_sample['knowledge'], orig_resp = test_sample['original_response'])

In [31]:
# epoch 10 seqlen 256
print('p:', test_sample['history'][-1])
print('k:', test_sample['knowledge'])
print('org:', test_sample['original_response'])
print('final prediction: ', targets)
print('expected_begin: ', test_sample["BEGIN"])
# print("expected_vrm: ", test_sample["VRM"])

p: Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.
k: Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).
org: Same and Increasingly, elements and principles of game design are also applied to other interactions
final prediction:  Hallucination,Entailment
expected_begin:  ['Hallucination', 'Entailment']


In [32]:
print(targets)

Hallucination,Entailment


In [33]:
# Epoch 10 seqlen 512
print('p:', test_sample['history'][-1])
print('k:', test_sample['knowledge'])
print('org:', test_sample['original_response'])
print('final prediction: ', targets)
# print('expected_vrm: ', test_sample["VRM"])
print("expected_begin: ", test_sample["BEGIN"])


p: Yes! I enjoy making sports and strategy games. I love the way players caninteract on a sports game in a simulated environment.
k: Increasingly, elements and principles of game design are also applied to other interactions, particularly virtual ones (see gamification).
org: Same and Increasingly, elements and principles of game design are also applied to other interactions
final prediction:  Hallucination,Entailment
expected_begin:  ['Hallucination', 'Entailment']


In [34]:
ground_truths = test_sample["BEGIN"]

### Evaluating the model 

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer

In [36]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(dataset['train']["BEGIN"])
num_labels = len(mlb.classes_)

In [37]:
mlb.classes_

array(['Entailment', 'Generic', 'Hallucination', 'Uncooperative'],
      dtype=object)

In [38]:
yhat = []
for i in range(len(dataset['test'])):
  n = []
  print("example", i)
  targets = infer(model, knowledge=dataset['test']['knowledge'][i], prompt = dataset['test']['history'][i][-1], orig_resp = dataset['test']['original_response'][i])
  targets = targets.split(",")
  for j in range(len(mlb.classes_)):
    if mlb.classes_[j] in targets:
      n.append(1)
    else:
      n.append(0)
  yhat.append(n)

example 0
example 1
example 2
example 3
example 4
example 5
example 6
example 7
example 8
example 9
example 10
example 11
example 12
example 13
example 14
example 15
example 16
example 17
example 18
example 19
example 20
example 21
example 22
example 23
example 24
example 25
example 26
example 27
example 28
example 29
example 30
example 31
example 32
example 33
example 34
example 35
example 36
example 37
example 38
example 39
example 40
example 41
example 42
example 43
example 44
example 45
example 46
example 47
example 48
example 49
example 50
example 51
example 52
example 53
example 54
example 55
example 56
example 57
example 58
example 59
example 60
example 61
example 62
example 63
example 64
example 65
example 66
example 67
example 68
example 69
example 70
example 71
example 72
example 73
example 74
example 75
example 76
example 77
example 78
example 79
example 80
example 81
example 82
example 83
example 84
example 85
example 86
example 87
example 88
example 89
example 90
example 9

In [39]:
yhat

[[1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 0,

In [40]:
y = []
for i in range(len(dataset['test'])):
  n = []
  exp = dataset['test']["BEGIN"][i]
  for j in range(len(mlb.classes_)):
    if mlb.classes_[j] in exp:
      n.append(1)
    else:
      n.append(0)
  y.append(n)
print(y)

[[0, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 1, 0], [0, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0

In [41]:
from sklearn.metrics import f1_score
f1 = []
for i in range(len(yhat)):
  macro_f1 = f1_score(yhat[i], y[i], average='macro')
  f1.append(macro_f1)
print(f1)
print("Macro-Averaged F1 Score:", sum(f1)/len(f1))

[0.7333333333333334, 0.5, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 1.0, 0.2, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.2, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 0.3333333333333333, 1.0, 0.7333333

In [42]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
f1 = []
accuracy = []
for i in range(len(yhat)):
  macro_f1 = f1_score(yhat[i], y[i], average='macro')
  f1.append(macro_f1)
  acc = accuracy_score(y[i], yhat[i])
  accuracy.append(acc)
print(f1)
print(accuracy)
print("Macro-Averaged F1 Score:", sum(f1)/len(f1))
print("Average accuracy", sum(accuracy)/len(accuracy))

[0.7333333333333334, 0.5, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 1.0, 0.2, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 1.0, 1.0, 1.0, 0.7333333333333334, 0.7333333333333334, 1.0, 0.2, 0.7333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7333333333333334, 0.3333333333333333, 1.0, 0.7333333

In [43]:
print("Average accuracy", sum(accuracy)/len(accuracy))

Average accuracy 0.9078129415089008


In [44]:
from sklearn.metrics import multilabel_confusion_matrix
confusion_matrix = multilabel_confusion_matrix(y, yhat)
for i, cm in enumerate(confusion_matrix):
    print(f"Confusion matrix for class {i + 1}:\n{cm}\n")

Confusion matrix for class 1:
[[ 158  702]
 [  46 2633]]

Confusion matrix for class 2:
[[3506    1]
 [  30    2]]

Confusion matrix for class 3:
[[ 762  216]
 [  73 2488]]

Confusion matrix for class 4:
[[3221  108]
 [ 129   81]]

